In [15]:
import pandas as pd
import numpy as np
import json

In [16]:
graph = "stexpanded"
embeddings_file = "results/embeddings/" + graph + "_lab_altlab_type_abs_comment_BAAI_bge-large-en-v1.5.json"
top_betweenness_file = "results/top_betweenness/" + graph + ".json"
graph_file = "data/triples_v2/" + graph + ".triples"

In [17]:
with open(graph_file, 'r') as f:
    for line in f:
        items = line.strip().split('###')
        if items[0] == '2239' or items[1] == '2239':
            print(line.strip())

1300###2239###23
4080###2239###23
2278###2239###23
3539###2239###23
352###2239###23
2217###2239###23
212###2239###23
2826###2239###23
744###2239###23
499###2239###23
22998###2239###23
23807###2239###23
25326###2239###23
704###2239###23
26847###2239###23
27539###2239###23
27943###2239###23
28060###2239###23
1020###2239###23
528###2239###23
7163###2239###23
28116###2239###23
2862###2239###23
5420###2239###23
1561###2239###23
41671###2239###23
16211###2239###23
45659###2239###23
3793###2239###23
5745###2239###23
47527###2239###23
2252###2239###23
49754###2239###23
263###2239###23
31457###2239###23
1556###2239###23
51630###2239###23
52235###2239###23
4733###2239###23
1733###2239###23
7057###2239###23
18555###2239###23
14052###2239###23
44984###2239###23
1261###2239###23
17###2239###23
1209###2239###23
36###2239###23
19674###2239###23
49545###2239###23
46447###2239###23
255###2239###23
43901###2239###23
148###2239###23
1###2239###23
938###2239###23
61###2239###23
27218###2239###23
1487###22

In [18]:
df = pd.read_json(embeddings_file)
df_T = df.T

In [19]:
df_T

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.032848,-0.064748,0.030845,0.027461,0.002914,0.001362,-0.016269,0.054001,-0.011309,-0.013344,...,-0.007416,-0.008869,0.002427,0.033333,-0.002463,-0.029383,0.041667,0.023747,-0.008002,-0.054726
2,-0.018349,-0.024229,-0.063716,0.015355,-0.048345,-0.002276,-0.031895,0.008005,-0.000620,0.038002,...,0.020007,-0.002332,0.018543,0.007513,0.017034,0.020338,-0.008677,0.016864,-0.011206,-0.045230
6,0.021354,0.025183,-0.004776,0.035773,-0.027300,0.016972,-0.055827,-0.018391,0.011529,0.033098,...,0.000795,-0.019874,-0.042116,-0.017028,0.048721,0.028457,0.085684,0.014370,0.001986,-0.030163
8,-0.008521,0.000420,-0.020345,0.023471,-0.013444,0.001813,-0.046061,0.001937,0.020917,0.011916,...,-0.005840,-0.052993,-0.037468,0.006375,0.023490,-0.002378,0.045731,0.002556,0.022744,-0.035521
9,0.036227,0.017489,-0.006952,-0.022873,-0.029047,-0.010171,-0.047689,0.003121,0.057789,0.022159,...,0.019363,0.000237,-0.033890,0.011580,-0.020793,0.016644,0.050162,0.017962,-0.011384,-0.024669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,0.000415,0.008703,-0.013847,-0.005123,-0.025955,-0.001055,0.020327,-0.013105,0.023191,0.018944,...,0.020520,-0.014834,-0.033848,-0.011127,0.019989,0.044828,0.026601,-0.017876,-0.018791,0.038948
26474,0.010162,0.014899,0.004704,-0.022322,-0.036034,0.025204,0.033319,-0.010035,0.014106,0.026672,...,0.025497,-0.024413,-0.025961,-0.014532,0.021326,0.018163,0.063446,-0.019200,-0.022384,0.031233
120379,0.029356,0.020848,-0.029341,0.000359,-0.033287,-0.013522,0.010909,0.030116,0.026685,0.026973,...,0.008612,-0.005447,-0.020129,0.026309,0.035329,0.034377,0.018843,-0.022818,0.013571,0.011612
121821,0.000787,0.022212,-0.001543,-0.005916,-0.016270,0.011212,0.001876,0.005813,0.008669,0.018673,...,0.036610,-0.016965,-0.025528,-0.009747,0.036980,0.032287,0.056192,-0.001534,-0.004540,0.029594


In [20]:
df_T.loc[2239]

KeyError: 2239

In [21]:
with open(top_betweenness_file, "r") as f:
    top_betweenness = json.load(f)

In [22]:
top_betweenness[0]

{'2239': 1.0,
 '1': 0.0,
 '52235': 0.0,
 '528': 0.0,
 '17': 0.0,
 '6675': 0.0,
 '1556': 0.0,
 '1561': 0.0,
 '30': 0.0,
 '8227': 0.0,
 '36': 0.0,
 '47670': 0.0,
 '37944': 0.0,
 '56': 0.0,
 '61': 0.0,
 '13886': 0.0,
 '11334': 0.0,
 '3143': 0.0,
 '76': 0.0,
 '38989': 0.0,
 '2126': 0.0,
 '1104': 0.0,
 '50769': 0.0,
 '27218': 0.0,
 '49754': 0.0,
 '45659': 0.0,
 '9309': 0.0,
 '7774': 0.0,
 '1120': 0.0,
 '1637': 0.0,
 '11878': 0.0,
 '6248': 0.0,
 '23662': 0.0,
 '112': 0.0,
 '5745': 0.0,
 '23157': 0.0,
 '3189': 0.0,
 '8310': 0.0,
 '38522': 0.0,
 '18555': 0.0,
 '637': 0.0,
 '4733': 0.0,
 '8842': 0.0,
 '74893': 0.0,
 '6286': 0.0,
 '2191': 0.0,
 '21650': 0.0,
 '148': 0.0,
 '26264': 0.0,
 '9368': 0.0,
 '6301': 0.0,
 '157': 0.0,
 '21662': 0.0,
 '7838': 0.0,
 '166': 0.0,
 '41640': 0.0,
 '2217': 0.0,
 '6314': 0.0,
 '171': 0.0,
 '91309': 0.0,
 '5302': 0.0,
 '1209': 0.0,
 '2233': 0.0,
 '704': 0.0,
 '1733': 0.0,
 '41671': 0.0,
 '3274': 0.0,
 '100043': 0.0,
 '2252': 0.0,
 '10956': 0.0,
 '3793': 0.0,
 '21

In [27]:
len(top_betweenness)

1164

In [30]:
num_error = 0
for top_b in top_betweenness:
    top_b = {int(k): v for k, v in top_b.items()}
    keys = list(top_b.keys())
    # print(keys)
    for key in keys:
        try:
            selected_rows = df_T.loc[key]
        except KeyError:
            num_error += 1
            print(f"Key '{key}' not found in the dataframe.")
            selected_rows = None
print(num_error)

# weights = np.array(list(top_betweenness[0].values()))
# print(weights)
# weighted_avg = np.average(selected_columns, axis=0, weights=weights)
# print(weighted_avg)

Key '2239' not found in the dataframe.
Key '17023' not found in the dataframe.
Key '52729' not found in the dataframe.
Key '27194' not found in the dataframe.
Key '60576' not found in the dataframe.
Key '45315' not found in the dataframe.
Key '22150' not found in the dataframe.
Key '107017' not found in the dataframe.
Key '45847' not found in the dataframe.
Key '28696' not found in the dataframe.
Key '7450' not found in the dataframe.
Key '83236' not found in the dataframe.
Key '106149' not found in the dataframe.
Key '14763' not found in the dataframe.
Key '73520' not found in the dataframe.
Key '55603' not found in the dataframe.
Key '2613' not found in the dataframe.
Key '70454' not found in the dataframe.
Key '52410' not found in the dataframe.
Key '61375' not found in the dataframe.
Key '78657' not found in the dataframe.
Key '69321' not found in the dataframe.
Key '47690' not found in the dataframe.
Key '42827' not found in the dataframe.
Key '9805' not found in the dataframe.
Ke

In [ ]:
list(top_betweenness[0].keys())[0]